# 데이터 크롤링

- 크롤링 패키지 selenium

In [5]:
# !pip install selenium

In [1]:
from selenium import webdriver

## JustWatch 메인 페이지

- 메인 페이지에서 가져올 수 있는 링크들 전부 가져오기

In [7]:
URL = 'https://www.justwatch.com/kr/%EB%8F%99%EC%98%81%EC%83%81%EC%84%9C%EB%B9%84%EC%8A%A4/netflix'

driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url=URL)

In [8]:
# 21904 개
links = []

- 스크롤 내리기

In [154]:
import time
for i in range(30):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)

- 스크롤 최대 1950개 호출 가능...

In [155]:
elements = driver.find_elements_by_xpath('//*[@id="base"]/div[3]/div/div[2]/div[1]/div/div/a')
len(elements)

1136

세부적으로 중복이 없는 연도, 장르를 구분하여 데이터를 분리하여 크롤링

In [156]:
# 1900 ~ 1993 : 1797
# 1994 ~ 2004 : 1881
# 2005 ~ 2009 : 1932
# 2010 ~ 2012 : 1894
# 2013 ~ 2014 : 2089 = 513(액션) + 1576(액션 -)
# 2014 : 1141
# 2015 : 1278
# 2016 : 1553
# 2017 : 1883
# 2018 ~ 2021 : 7598 = 1462(액션) + 1834(-, 애, 코) + 1834(-, 범,다,판,역,공 +) + 1332 (-, 드라마) + 1136
for element in elements:
    links.append(element.get_attribute('href'))
len(links)

24720

In [168]:
df_links = pd.DataFrame(links, columns = ["links"])
df_links = df_links.drop_duplicates(keep='first')    # 중복 제거
df_links.to_csv("links.csv", index=False)            # 데이터 csv 타입으로 저장

In [9]:
driver.close()

In [2]:
import pandas as pd

df_links = pd.read_csv("./links.csv")
movies = df_links[df_links.links.apply(lambda x : False if "TV-" in x else True)]

In [3]:
movies.iloc[4760]["links"]z

'https://www.justwatch.com/kr/%EC%98%81%ED%99%94/jeonhagsaeng'

In [ ]:
//*[@id="base"]/div[2]/div/div[2]/div[2]/div[6]/div[1]
//*[@id="base"]/div[2]/div/div[2]/div[2]/div[5]/div[1]

In [15]:
for i in range(1, 10):
    print(i)

1
2
3
4
5
6
7
8
9


In [30]:
with open(f"error_log{p}.txt", "a") as file:


In [4]:
from selenium import webdriver
import pandas as pd
# 영화,
# 애니, 드라마, 예능, tv 시리즈 구분하기...

options = webdriver.ChromeOptions()
options.add_argument("headless")

num = len(movies)

p = 6000
with open(f"error_log{p}.txt", "w") as file:

    driver = webdriver.Chrome(executable_path='chromedriver', options=options)

    for times in range(6, 7): # 0(0 ~ 999) 1(1000 ~ 1999)
        data_list = []
        for link in movies["links"][p:p+1000]:
            p += 1
            print(round(p / num * 100, 2), "%", p, end="")
            file.write(f'{round(p / num * 100, 2)}%, {p}, ')
            driver.get(url=link)
            data = {}
            # 제목, 개봉일 가져오기
            element = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]')
            data["제목"] = element.find_element_by_xpath("div/h1").text
            try: # 한국영화
                data["원제"] = element.find_element_by_xpath("h3").text.split("원제: ")[1]
            except:
                print("한국영화 원제 오류 :", p, ", ", end="")
                file.write(f"한국영화 원제 오류, ")
            data["개봉일"] = element.find_element_by_xpath("div/span").text

            # (필터) 스트리밍, 대여, 구매
            element = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[1]/div[2]/div')
            OTTs = element.find_elements_by_xpath('div[position() > 1]')

            for ott in OTTs:
                try: # 398 프랑켄슈타인
                    streams = ott.find_elements_by_xpath('div')
                    
                    temp = streams[0].text
                    data[temp] = (
                        [x.get_attribute('title') for x in streams[1].find_elements_by_xpath('div/div/a/picture/img')],
                        streams[1].text.split()
                    )
                    ls = []
                    for i in range(len(data[temp][0])):
                        ls.append((data[temp][0][i], data[temp][1][i*2:i*2+2]))
                    data[temp] = ls
                except:
                    print("스트리밍 빈공간 오류 :", p, ", ", end="")
                    file.write(f"스트리밍 빈공간 오류, ")

            # 세부 특징
            element = driver.find_elements_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]')[-1]
            contents = element.find_elements_by_xpath("div")
            points = contents[0].find_elements_by_xpath("div")
            for point in points:
                data[point.text.split("\n")[0]] = point.text.split("\n")[1:]
            
            if contents[1].text == "출연진":
                data[contents[1].text] = contents[2].text.split('\n')[::2]
                try: # 시놉시스가 없는 경우 10020
                    data[contents[3].text.split('\n')[0]] = contents[3].text.split('\n')[1]
                except:
                    print("시놉시스 없는 오류 :", p, ", ", end="")
                    file.write(f"시놉시스 없는 오류, ")
            else : # 출연진 없는경우 635
                try:
                    data[contents[2].text.split('\n')[0]] = contents[2].text.split('\n')[1]
                except:
                    print("출현진, 시놉시스 없는 오류 :", p, ", ", end="")
                    file.write(f"출현진, 시놉시스 없는 오류, ")
            
            # 이미지 가져오기
            try: # 이미지 없음 635
                element = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[1]/div/aside/div[2]/div/picture/source[1]')    
                data["이미지"] = element.get_attribute('srcset').split(',')[0]
            except:
                print("이미지 없는 오류 :", p, ", ", end="")
                file.write(f"이미지 없는 오류, ")
            print()
            file.write("\n")
            
            data_list.append(data)
            
        df = pd.DataFrame(data_list)
        df.to_csv(f"movie_list_{times}.csv", index=False)

    driver.close()


36.73 % 6001
36.74 % 6002
36.74 % 6003한국영화 원제 오류 : 6003 , 
36.75 % 6004
36.75 % 6005
36.76 % 6006
36.77 % 6007
36.77 % 6008한국영화 원제 오류 : 6008 , 
36.78 % 6009
36.79 % 6010
36.79 % 6011
36.8 % 6012한국영화 원제 오류 : 6012 , 
36.8 % 6013
36.81 % 6014
36.82 % 6015
36.82 % 6016
36.83 % 6017한국영화 원제 오류 : 6017 , 
36.83 % 6018한국영화 원제 오류 : 6018 , 
36.84 % 6019
36.85 % 6020
36.85 % 6021
36.86 % 6022한국영화 원제 오류 : 6022 , 
36.86 % 6023
36.87 % 6024한국영화 원제 오류 : 6024 , 
36.88 % 6025
36.88 % 6026
36.89 % 6027한국영화 원제 오류 : 6027 , 
36.9 % 6028한국영화 원제 오류 : 6028 , 
36.9 % 6029
36.91 % 6030
36.91 % 6031
36.92 % 6032한국영화 원제 오류 : 6032 , 
36.93 % 6033
36.93 % 6034
36.94 % 6035
36.94 % 6036한국영화 원제 오류 : 6036 , 
36.95 % 6037한국영화 원제 오류 : 6037 , 
36.96 % 6038
36.96 % 6039
36.97 % 6040
36.98 % 6041
36.98 % 6042한국영화 원제 오류 : 6042 , 
36.99 % 6043
36.99 % 6044한국영화 원제 오류 : 6044 , 
37.0 % 6045
37.01 % 6046한국영화 원제 오류 : 6046 , 
37.01 % 6047
37.02 % 6048
37.02 % 6049
37.03 % 6050한국영화 원제 오류 : 6050 , 
37.04 % 6051
37.04 % 6052한국영화 원제 오류

In [15]:
driver.get(url=link)

In [16]:
element = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]')

In [ ]:
//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]
//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]

In [12]:


df = pd.DataFrame(data_list)
df.to_csv(f"movie_list_{times}.csv", index=False)

In [9]:
data

{'제목': 'Rebecca',
 '개봉일': '(1952)',
 '대여': [('wavve', ['₩1,000', 'HD'])],
 '구매': [('Naver Store', ['₩1,000', 'HD'])],
 '평점': [],
 '장르': ['드라마'],
 '재생 시간': ['2시간 9분'],
 '시놉시스': "TESSIE AGANA: The nation's child sweetheart."}

In [7]:
def scraper(link):
    print("크롤링중")
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(executable_path='chromedriver', options=options)

    driver.get(url=link)
    data = {}
    # 제목, 개봉일 가져오기
    element = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]')
    data["제목"] = element.find_element_by_xpath("div/h1").text
    try: # 한국영화
        data["원제"] = element.find_element_by_xpath("h3").text.split("원제: ")[1]
    except:
        pass
    data["개봉일"] = element.find_element_by_xpath("div/span").text

        # (필터) 스트리밍, 대여, 구매
    element = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[1]/div[2]/div')
    OTTs = element.find_elements_by_xpath('div[position() > 1]')

    for ott in OTTs:
        try: # 398 프랑켄슈타인
            streams = ott.find_elements_by_xpath('div')
            
            temp = streams[0].text
            data[temp] = (
                [x.get_attribute('title') for x in streams[1].find_elements_by_xpath('div/div/a/picture/img')],
                streams[1].text.split()
            )
            ls = []
            for i in range(len(data[temp][0])):
                ls.append((data[temp][0][i], data[temp][1][i*2:i*2+2]))
            data[temp] = ls
        except:
            pass

    # 세부 특징
    element = driver.find_elements_by_xpath('//*[@id="base"]/div[2]/div/div[2]/div[2]/div/div[1]')[-1]
    contents = element.find_elements_by_xpath("div")
    points = contents[0].find_elements_by_xpath("div")
    for point in points:
        data[point.text.split("\n")[0]] = point.text.split("\n")[1:]
    
    if contents[1].text == "출연진":
        data[contents[1].text] = contents[2].text.split('\n')[::2]
        try: # 시놉시스가 없는 경우 10020
            data[contents[3].text.split('\n')[0]] = contents[3].text.split('\n')[1]
        except:
            pass
    else : # 출연진 없는경우 635
        try:
            data[contents[2].text.split('\n')[0]] = contents[2].text.split('\n')[1]
        except:
            pass
    
    # 이미지 가져오기
    try: # 이미지 없음 635
        element = driver.find_element_by_xpath('//*[@id="base"]/div[2]/div/div[1]/div/aside/div[2]/div/picture/source[1]')    
        data["이미지"] = element.get_attribute('srcset').split(',')[0]
    except:
        pass
    driver.close()
    
    return data

In [3]:

df_links = pd.read_csv("./links.csv")
movies = df_links[df_links.links.apply(lambda x : False if "TV-" in x else True)]
link_list = movies.reset_index(drop = True)

In [1]:
# !pip install nest_asyncio

In [8]:
import asyncio
from concurrent.futures.thread import ThreadPoolExecutor
import nest_asyncio

# nest_asyncio.apply()

from selenium import webdriver

executor = ThreadPoolExecutor(10)

def scrape(link, *, loop):
    print(link)
    loop.run_in_executor(executor, scraper, link)

data_list = []
loop = asyncio.get_event_loop()
for link in link_list["links"][4000:4100]:
    data_list.append(scrape(link, loop=loop))

loop.run_until_complete(asyncio.gather(*asyncio.all_tasks(loop)))

https://www.justwatch.com/kr/%EC%98%81%ED%99%94/oteobsi
크롤링중https://www.justwatch.com/kr/%EC%98%81%ED%99%94/the-secret

크롤링중
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/teuripeul-egseu-2-negseuteu-rebel
크롤링중https://www.justwatch.com/kr/%EC%98%81%ED%99%94/tom-and-jerry-blast-off-to-mars

크롤링중
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/kaendi
크롤링중https://www.justwatch.com/kr/%EC%98%81%ED%99%94/keeping-up-with-the-steins

크롤링중
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/the-secret-life-of-bees
크롤링중
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/all-about-steve
크롤링중https://www.justwatch.com/kr/%EC%98%81%ED%99%94/solstice

크롤링중
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/weoteo-rilriseu
크롤링중
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/kaedilrag-rekodeu
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/eighteen
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/chicken-little
https://www.justwatch.com/kr/%EC%98%81%ED%99%94/i-now-pronounce-you-chuck-and-larry
https://www.just

[]

크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중
크롤링중


In [6]:
pd.DataFrame(data_list)

,0
0,None
1,None
2,None
3,None
4,None
...,...
95,None
96,None
97,None
98,None


In [243]:
//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]
//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]
//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]
//*[@id="base"]/div[2]/div/div[2]/div[2]/div[1]/div[1]

In [247]:
df.isna().sum()

제목        0
원제        1
개봉일       0
스트리밍     40
구매       17
평점        0
장르        0
재생 시간     0
감독        0
출연진       0
시놉시스      0
이미지       0
대여       38
dtype: int64

,제목,원제,개봉일,스트리밍,구매,평점,장르,재생 시간,감독,출연진,시놉시스,이미지,대여
0,나 홀로 집에,Home Alone,(1990),"[(Disney Plus, [정액제, 4K])]","[(wavve, [₩5,500, HD]), (Naver Store, [₩5,500,...","[82%, 7.6 (511k)]","[코미디 , 가족]",[1시간 43분],[Chris Columbus],"[Macaulay Culkin, Joe Pesci, Daniel Stern, Joh...",크리스마스 시즌의 시카고. 말썽꾸러기 케빈(맥컬리 컬킨)은 자기의 치즈 피자를 먹은...,https://images.justwatch.com/poster/240463286/...,NaN
1,크리스마스 대소동,National Lampoon's Christmas Vacation,(1989),NaN,"[(Naver Store, [₩4,950, HD])]","[82%, 7.6 (173k)]","[코미디 , Made in Europe]",[1시간 37분],[Jeremiah S. Chechik],"[Chevy Chase, Beverly D'Angelo, Juliette Lewis...","클라크 오스왈드(Clark Wilhelm Griswold, Jr.: 체비 체이스 분...",https://images.justwatch.com/poster/248460937/...,"[(Naver Store, [₩1,300, HD])]"
2,고스트 버스터즈,Ghostbusters,(1984),"[(Netflix, [정액제, 4K])]","[(Naver Store, [₩5,000, HD])]","[84%, 7.8 (373k)]","[판타지 , SF , 액션 , 코미디]",[1시간 41분],[Ivan Reitman],"[Bill Murray, Dan Aykroyd, Sigourney Weaver, H...",초심리를 연구하는 뉴욕의 괴짜교수 피터 밴크맨(빌 머레이)와 레이몬드 스탠드(댄 애...,https://images.justwatch.com/poster/19416696/s...,"[(Naver Store, [₩1,300, HD])]"
3,고스트 버스터즈 2,Ghostbusters II,(1989),"[(wavve, [정액제, HD]), (Watcha, [정액제, HD])]","[(Naver Store, [₩5,000, HD]), (wavve, [₩5,500,...","[74%, 6.6 (195k)]","[판타지 , SF , 액션 , 코미디]",[1시간 48분],[Ivan Reitman],"[Bill Murray, Dan Aykroyd, Sigourney Weaver, H...",고스트 버스터즈 4인방은 악마 '고저'로부터 세계를 구한 후 순탄한 생활을 하지 못...,https://images.justwatch.com/poster/20537077/s...,"[(wavve, [₩1,300, HD]), (Naver Store, [₩1,300,..."
4,멋진 인생,It's a Wonderful Life,(1946),"[(Plex, [무료, HD])]","[(Naver Store, [₩1,000])]","[92%, 8.6 (428k)]","[가족 , 판타지 , 드라마]",[2시간 10분],[Frank Capra],"[James Stewart, Donna Reed, Lionel Barrymore, ...",조지 베일리는 베드폴드 마을 사람들을 위해 평생을 봉사하며 살아온 인물. 그는 항상...,https://images.justwatch.com/poster/244992618/...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,에이리언 2,Aliens,(1986),"[(Netflix, [정액제, HD]), (Disney Plus, [정액제, HD])]","[(wavve, [₩5,500, HD]), (Naver Store, [₩5,500,...","[88%, 8.3 (685k)]","[스릴러 , 공포 , SF , 액션]",[2시간 17분],[James Cameron],"[Sigourney Weaver, Carrie Henn, Michael Biehn,...",에일리언과의 싸움에서 간신히 살아남은 리플리는 잠든 채로 57년을 떠돌다 구조선에 ...,https://images.justwatch.com/poster/253114261/...,NaN
96,빅,Big,(1988),"[(Disney Plus, [정액제, HD])]","[(wavve, [₩5,500, HD]), (Naver Store, [₩5,500,...","[79%, 7.3 (209k)]","[판타지 , 드라마 , 로맨스 , 코미디]",[1시간 44분],[Penny Marshall],"[Tom Hanks, Elizabeth Perkins, Robert Loggia, ...",13세 난 개구쟁이 조쉬는 어느날 축제에 놀러갔다가 '졸타'라는 기계에 어른이 되고...,https://images.justwatch.com/poster/240819275/...,NaN
97,에어플레인,Airplane!,(1980),NaN,"[(wavve, [₩5,500, HD])]","[82%, 7.7 (225k)]",[코미디],[1시간 28분],"[Jim Abrahams , Jerry Zucker , David Zucker]","[Robert Hays, Julie Hagerty, Leslie Nielsen, K...",택시 기사인 테드 스트라이커(Ted Striker: 로버트 헤이스 분)와 스튜어디스...,https://images.justwatch.com/poster/8715463/s2...,"[(wavve, [₩1,500, HD])]"
98,영 프랑켄슈타인,Young Frankenstein,(1974),"[(wavve, [정액제, HD]), (Watcha, [정액제, HD])]","[(Naver Store, [₩1,500, HD])]","[86%, 8.0 (152k)]",[코미디],[1시간 46분],[Mel Brooks],"[Gene Wilder, Teri Garr, Marty Feldman, Peter ...",공포물 을 멜 브룩스 감독이 패러디하여 코미디로 만든 작품. 자욱한 안개와 폭풍이 ...,https://images.justwatch.com/poster/21131754/s...,"[(wavve, [₩1,500, HD])]"


In [203]:
driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=96.0.4664.110)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D56903+2517251]
	Ordinal0 [0x00CEF8E1+2095329]
	Ordinal0 [0x00BF2710+1058576]
	Ordinal0 [0x00BE76A4+1013412]
	Ordinal0 [0x00BDBC01+965633]
	Ordinal0 [0x00C3C180+1360256]
	Ordinal0 [0x00C364A7+1336487]
	Ordinal0 [0x00C127A7+1189799]
	Ordinal0 [0x00C13609+1193481]
	GetHandleVerifier [0x00EE5904+1577972]
	GetHandleVerifier [0x00F90B97+2279047]
	GetHandleVerifier [0x00DE6D09+534521]
	GetHandleVerifier [0x00DE5DB9+530601]
	Ordinal0 [0x00CF4FF9+2117625]
	Ordinal0 [0x00CF98A8+2136232]
	Ordinal0 [0x00CF99E2+2136546]
	Ordinal0 [0x00D03541+2176321]
	BaseThreadInitThunk [0x761AFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77577A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77577A6E+238]
